In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook


## 预处理

In [85]:
train_x=pd.read_csv('data/train_weather_utf8.csv')
train_y=pd.read_csv('data/train_rice_utf8.csv')

In [36]:
train_x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118341 entries, 2922 to 131489
Data columns (total 17 columns):
区县id             118341 non-null object
站名id             118341 non-null int64
年份               118341 non-null int64
月份               118341 non-null int64
日期               118341 non-null int64
日照时数（单位：h)       118341 non-null object
02时风向            118341 non-null object
08时风向            118341 non-null object
14时风向            118341 non-null object
20时风向            118341 non-null object
日平均风速(单位：m/s)    118341 non-null object
日降水量（mm）         107887 non-null object
日最高温度（单位：℃）      118341 non-null float64
日最低温度（单位：℃）      118341 non-null float64
日平均温度（单位：℃）      118341 non-null object
日相对湿度（单位：%）      118341 non-null object
日平均气压（单位：hPa）    118341 non-null object
dtypes: float64(2), int64(4), object(11)
memory usage: 16.3+ MB


In [9]:
train_x.head()

,区县id,站名id,年份,月份,日期,日照时数（单位：h),02时风向,08时风向,14时风向,20时风向,日平均风速(单位：m/s),日降水量（mm）,日最高温度（单位：℃）,日最低温度（单位：℃）,日平均温度（单位：℃）,日相对湿度（单位：%）,日平均气压（单位：hPa）
0,county25,1,2015,1,1,6.3,NNW,N,NNE,S,1.8,0,14.1,7.2,9.1,43,980.6
1,county25,1,2015,1,2,8.1,SSE,S,SSW,S,1.3,0,18.3,1.6,7.5,64,977.7
2,county25,1,2015,1,3,4.3,S,SSE,S,SSE,2,0,18.9,2.4,9.4,66,973.1
3,county25,1,2015,1,4,0,WNW,SSW,S,SSE,2.2,0.3,14.9,9.0,11.5,76,966.7
4,county25,1,2015,1,5,0,SSE,S,SSW,C,2.8,0.8,16.8,11.6,15.5,91,963.9


In [86]:
# 保留种植成熟时间
month_index=train_x['月份'].apply(lambda x: x in np.arange(3,8))
train_x=train_x[month_index]

In [70]:
train_x[train_x['站名id']==1]['区县id'].unique(),train_x[train_x['站名id']==2]['区县id'].unique()

(array(['county25', 'county69', 'county14', 'county24', 'county12',
        'county13', 'county22', 'county18', 'county21', 'county19',
        'county15', 'county16', 'county20', 'county23', 'county62',
        'county61', 'county59', 'county58', 'county60', 'county70',
        'county68', 'county67', 'county71', 'county74', 'county73',
        'county72', 'county76', 'county75', 'county78', 'county10',
        'county8', 'county11', 'county9', 'county17', 'county28',
        'county26', 'county27', 'county81', 'county32', 'county64',
        'county66', 'county65', 'county63', 'county57', 'county51',
        'county52', 'county55', 'county56', 'county53', 'county87',
        'county54', 'county3', 'county4', 'county5', 'county2', 'county6',
        'county79', 'county77', 'county80', 'county43', 'county45',
        'county44', 'county31', 'county29', 'county30', 'county85',
        'county88', 'county86', 'county82', 'county83', 'county84',
        'county38', 'county1', 'county7', '

In [87]:
predict_x=pd.read_csv('data/testA.csv', header=None)   # 获取要预测的县

In [88]:
predict_id=train_x['区县id'].apply(lambda x: x in predict_x.values)
train_x=train_x[predict_id]

In [89]:
predict_id=train_x['站名id'].apply(lambda x: x is 1)
train_x=train_x[predict_id]

In [6]:
train_x['区县id'].apply(lambda x: x in predict_x.values).value_counts()

True    122724
Name: 区县id, dtype: int64

## 缺失值

In [248]:
train_x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49572 entries, 2981 to 131336
Data columns (total 17 columns):
区县id             49572 non-null object
站名id             49572 non-null int64
年份               49572 non-null int64
月份               49572 non-null int64
日期               49572 non-null int64
日照时数（单位：h)       49572 non-null float64
02时风向            49571 non-null object
08时风向            49572 non-null object
14时风向            49572 non-null object
20时风向            49572 non-null object
日平均风速(单位：m/s)    49572 non-null float64
日降水量（mm）         49572 non-null float64
日最高温度（单位：℃）      49572 non-null float64
日最低温度（单位：℃）      49572 non-null float64
日平均温度（单位：℃）      49572 non-null float64
日相对湿度（单位：%）      49572 non-null float64
日平均气压（单位：hPa）    49572 non-null float64
dtypes: float64(8), int64(4), object(5)
memory usage: 8.1+ MB


In [91]:
train_x.replace('/',np.nan,inplace=True)

In [90]:
train_x.replace('*',np.nan,True)

In [251]:
# 查看缺失值
nan=train_x.isnull().sum()
nan_columns=nan[nan!=0]
nan_columns_index=nan_columns.index
nan_columns

02时风向    1
dtype: int64

In [169]:
train_x['日照时数（单位：h)'].fillna(0,inplace=True)
train_x['日照时数（单位：h)']=train_x['日照时数（单位：h)'].apply(pd.to_numeric)

In [166]:
nan_index=np.where(train_x['日平均风速(单位：m/s)'].isnull())[0]
for j in nan_index:
    mean=np.mean(train_x['日平均风速(单位：m/s)'].iloc[j-10:j].values)
    train_x['日平均风速(单位：m/s)'].iloc[j]=mean

[7423]
1.54


e:\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [ ]:
train_x['日平均风速(单位：m/s)']=train_x['日平均风速(单位：m/s)'].apply(pd.to_numeric)

In [189]:
train_x['日降水量（mm）'].fillna(0,inplace=True)
train_x['日降水量（mm）']=train_x['日平均风速(单位：m/s)'].apply(pd.to_numeric)

In [178]:
nan_index=np.where(train_x['日平均温度（单位：℃）'].isnull())[0]
train_x['日平均温度（单位：℃）']=train_x['日平均温度（单位：℃）'].apply(pd.to_numeric)
for j in nan_index:
    mean=np.mean(train_x['日平均温度（单位：℃）'].iloc[j-10:j].values)
    train_x['日平均温度（单位：℃）'].iloc[j]=mean

e:\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [181]:
nan_index=np.where(train_x['日相对湿度（单位：%）'].isnull())[0]
train_x['日相对湿度（单位：%）']=train_x['日相对湿度（单位：%）'].apply(pd.to_numeric)
for j in nan_index:
    mean=np.mean(train_x['日相对湿度（单位：%）'].iloc[j-10:j].values)
    train_x['日相对湿度（单位：%）'].iloc[j]=mean

e:\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [183]:
nan_index=np.where(train_x['日平均气压（单位：hPa）'].isnull())[0]
train_x['日平均气压（单位：hPa）']=train_x['日平均气压（单位：hPa）'].apply(pd.to_numeric)
for j in nan_index:
    mean=np.mean(train_x['日平均气压（单位：hPa）'].iloc[j-10:j].values)
    train_x['日平均气压（单位：hPa）'].iloc[j]=mean

e:\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


## 特征处理

In [236]:
direction_columns=[]

for i in ['02','08','14','20']:
    for j in 'ESWN':
        direction_columns.append(i+j)

train_x_direction=pd.DataFrame(columns=direction_columns,index=train_x.index)

In [246]:
train_x

,区县id,站名id,年份,月份,日期,日照时数（单位：h),02时风向,08时风向,14时风向,20时风向,日平均风速(单位：m/s),日降水量（mm）,日最高温度（单位：℃）,日最低温度（单位：℃）,日平均温度（单位：℃）,日相对湿度（单位：%）,日平均气压（单位：hPa）
2981,county14,1,2015,3,1,0.8,NNW,N,NNE,WNW,2.0,2.0,11.2,5.1,8.0,68.0,1002.9
2982,county14,1,2015,3,2,0.0,NE,WNW,W,W,0.7,0.7,9.4,7.1,8.0,90.0,992.9
2983,county14,1,2015,3,3,0.0,NNE,NNW,ENE,WNW,0.8,0.8,12.7,8.1,9.9,90.0,993.3
2984,county14,1,2015,3,4,0.0,NNW,WNW,NNE,NW,1.3,1.3,11.5,6.8,8.7,88.0,993.3
2985,county14,1,2015,3,5,0.0,NNE,NNE,NNE,N,1.7,1.7,8.7,7.1,7.5,85.0,995.6
2986,county14,1,2015,3,6,0.0,NNE,NE,NNE,NE,2.3,2.3,8.8,5.7,7.0,78.0,998.1
2987,county14,1,2015,3,7,0.0,NNW,NW,NNE,NNE,0.6,0.6,10.1,6.7,8.1,87.0,995.5
2988,county14,1,2015,3,8,0.0,NE,NE,NE,C,0.9,0.9,13.8,8.1,9.8,91.0,995.1
2989,county14,1,2015,3,9,0.0,NE,S,NE,NNE,1.4,1.4,15.7,10.5,12.6,80.0,997.8
2990,county14,1,2015,3,10,0.0,NNE,NW,NNE,NE,1.4,1.4,13.8,10.1,11.4,84.0,1000.2


In [243]:
for i in tqdm_notebook(train_x.index):
    train_x_direction_list=[]
    for j in ['02时风向','08时风向','14时风向','20时风向']:
        for z in 'ESWN':
            if pd.isna(train_x[j].loc[i]):
                train_x_direction_list.append(np.nan)
                continue
            if z in train_x[j].loc[i]:
                train_x_direction_list.append(1)
            else:
                train_x_direction_list.append(0)
    train_x_direction.loc[i,:]=train_x_direction_list
#     train_x.loc[i]
train_x_direction

,02E,02S,02W,02N,08E,08S,08W,08N,14E,14S,14W,14N,20E,20S,20W,20N
2981,0,0,1,1,0,0,0,1,1,0,0,1,0,0,1,1
2982,1,0,0,1,0,0,1,1,0,0,1,0,0,0,1,0
2983,1,0,0,1,0,0,1,1,1,0,0,1,0,0,1,1
2984,0,0,1,1,0,0,1,1,1,0,0,1,0,0,1,1
2985,1,0,0,1,1,0,0,1,1,0,0,1,0,0,0,1
2986,1,0,0,1,1,0,0,1,1,0,0,1,1,0,0,1
2987,0,0,1,1,0,0,1,1,1,0,0,1,1,0,0,1
2988,1,0,0,1,1,0,0,1,1,0,0,1,0,0,0,0
2989,1,0,0,1,0,1,0,0,1,0,0,1,1,0,0,1
2990,1,0,0,1,0,0,1,1,1,0,0,1,1,0,0,1


In [345]:
train_x_direction.to_csv('data/train_weather_direction.csv')

In [343]:
index=np.where(train_x_direction.isnull())[0]
train_x_direction['02E'].iloc[index[1]]=train_x_direction['02E'].value_counts().index[0]
train_x_direction['02S'].iloc[index[2]]=train_x_direction['02S'].value_counts().index[0]
train_x_direction['02W'].iloc[index[3]]=train_x_direction['02W'].value_counts().index[0]
train_x_direction['02N'].iloc[index[1]]=train_x_direction['02N'].value_counts().index[0]


e:\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [30]:
county=train_x['区县id'].unique()

In [31]:
# 分离id
def county_split(string):
    string_=string.replace('county','')
    value=int(string_)
    return value

split=np.vectorize(county_split)
id_=split(county)
id_=np.sort(id_,)

In [357]:
train_x.drop(columns=['02时风向','08时风向','14时风向','20时风向'],inplace=True)

In [377]:
train_x=pd.concat([train_x,train_x_direction],axis=1)

In [384]:
del train_x_direction

In [33]:
countyid=train_x['区县id']
yearid=train_x['年份']

In [410]:
train_x.drop(columns=['区县id','站名id','年份','月份','日期'],inplace=True)

In [415]:
train_x_array=np.array([],dtype=np.float)

In [423]:
train_x=train_x.apply(pd.to_numeric, errors='ignore')

In [432]:
year=[2015,2016,2017]
for i in tqdm_notebook(id_):
    county='county'+str(i)
    for j in year:
        index=np.intersect1d(np.where(countyid==county)[0],np.where(yearid==j)[0])
        if len(train_x_array)!=0:
            train_x_array=np.vstack((train_x_array,np.ravel(train_x.iloc[index])))
        else:
            train_x_array=np.ravel(train_x.iloc[index])

In [437]:
# train_weather_direction+train_weather_modified
np.save("data/modified_direction.npy",train_x_array) 

In [448]:
train_y_array=train_y.drop(columns=['区县id','2015年晚稻','2016年晚稻','2017年晚稻'])
train_y_array=np.ravel(train_y_array)
np.save('data/rice.npy',train_y_array)

- 抽取预测数据

In [48]:
train_x=pd.read_csv('data/train_weather_modified.csv',index_col=0)
train_x_direction=pd.read_csv('data/train_weather_direction.csv',index_col=0)
train_y_array=np.load("data/rice.npy")

In [49]:
train_x=pd.concat([train_x,train_x_direction],axis=1)
train_x.drop(columns=['区县id','站名id','年份','月份','日期'],inplace=True)

In [51]:
train_x

,日照时数（单位：h),日平均风速(单位：m/s),日降水量（mm）,日最高温度（单位：℃）,日最低温度（单位：℃）,日平均温度（单位：℃）,日相对湿度（单位：%）,日平均气压（单位：hPa）,02E,02S,...,08W,08N,14E,14S,14W,14N,20E,20S,20W,20N
2981,0.8,2.0,2.0,11.2,5.1,8.0,68.0,1002.9,0,0,...,0,1,1,0,0,1,0,0,1,1
2982,0.0,0.7,0.7,9.4,7.1,8.0,90.0,992.9,1,0,...,1,1,0,0,1,0,0,0,1,0
2983,0.0,0.8,0.8,12.7,8.1,9.9,90.0,993.3,1,0,...,1,1,1,0,0,1,0,0,1,1
2984,0.0,1.3,1.3,11.5,6.8,8.7,88.0,993.3,0,0,...,1,1,1,0,0,1,0,0,1,1
2985,0.0,1.7,1.7,8.7,7.1,7.5,85.0,995.6,1,0,...,0,1,1,0,0,1,0,0,0,1
2986,0.0,2.3,2.3,8.8,5.7,7.0,78.0,998.1,1,0,...,0,1,1,0,0,1,1,0,0,1
2987,0.0,0.6,0.6,10.1,6.7,8.1,87.0,995.5,0,0,...,1,1,1,0,0,1,1,0,0,1
2988,0.0,0.9,0.9,13.8,8.1,9.8,91.0,995.1,1,0,...,0,1,1,0,0,1,0,0,0,0
2989,0.0,1.4,1.4,15.7,10.5,12.6,80.0,997.8,1,0,...,0,0,1,0,0,1,1,0,0,1
2990,0.0,1.4,1.4,13.8,10.1,11.4,84.0,1000.2,1,0,...,1,1,1,0,0,1,1,0,0,1


In [54]:
# 预测数据集
predict_x_array=np.array([])
year=[2018]
for i in tqdm_notebook(id_):
    county='county'+str(i)
    for j in year:
        index=np.intersect1d(np.where(countyid==county)[0],np.where(yearid==j)[0])
        if len(predict_x_array)!=0:
            predict_x_array=np.vstack((predict_x_array,np.ravel(train_x.iloc[index])))
        else:
            predict_x_array=np.ravel(train_x.iloc[index])

## 特征处理1

In [3]:
train_x

,区县id,站名id,年份,月份,日期,日照时数（单位：h),日平均风速(单位：m/s),日降水量（mm）,日最高温度（单位：℃）,日最低温度（单位：℃）,日平均温度（单位：℃）,日相对湿度（单位：%）,日平均气压（单位：hPa）
2981,county14,1,2015,3,1,0.8,2.0,2.0,11.2,5.1,8.0,68.0,1002.9
2982,county14,1,2015,3,2,0.0,0.7,0.7,9.4,7.1,8.0,90.0,992.9
2983,county14,1,2015,3,3,0.0,0.8,0.8,12.7,8.1,9.9,90.0,993.3
2984,county14,1,2015,3,4,0.0,1.3,1.3,11.5,6.8,8.7,88.0,993.3
2985,county14,1,2015,3,5,0.0,1.7,1.7,8.7,7.1,7.5,85.0,995.6
2986,county14,1,2015,3,6,0.0,2.3,2.3,8.8,5.7,7.0,78.0,998.1
2987,county14,1,2015,3,7,0.0,0.6,0.6,10.1,6.7,8.1,87.0,995.5
2988,county14,1,2015,3,8,0.0,0.9,0.9,13.8,8.1,9.8,91.0,995.1
2989,county14,1,2015,3,9,0.0,1.4,1.4,15.7,10.5,12.6,80.0,997.8
2990,county14,1,2015,3,10,0.0,1.4,1.4,13.8,10.1,11.4,84.0,1000.2


In [5]:
train_x_direction=pd.read_csv('data/train_weather_direction.csv',index_col=0)
train_x=pd.read_csv('data/train_weather_modified.csv',index_col=0)
countyid=train_x['区县id']
yearid=train_x['年份']
train_x.drop(columns=['区县id','站名id','年份','月份','日期'],inplace=True)
train_x=pd.concat([train_x,train_x_direction],axis=1)
train_x=train_x.apply(pd.to_numeric, errors='ignore')
del train_x_direction

In [13]:
county=countyid.unique()
# 分离id
def county_split(string):
    string_=string.replace('county','')
    value=int(string_)
    return value

split=np.vectorize(county_split)
id_=split(county)
id_=np.sort(id_,)

In [12]:
# 一年天数
len(train_x[countyid=='county14'])/4 

153.0

In [80]:
# 时间滑动窗口 取平均 长度为n

def slide(n):
    train_x_array=np.array([],dtype=np.float)
    train_x_slide_array=np.array([],dtype=np.float)
    year=[2015,2016,2017]
    for i in tqdm_notebook(id_):
        county='county'+str(i)
        for j in year:
            index=np.intersect1d(np.where(countyid==county)[0],np.where(yearid==j)[0])
            train_x_index=train_x.iloc[index]
            for m in range(153-n+1):
                train_x_slide=np.mean(train_x_index.iloc[m:m+n-1].values,axis=0)
                if len(train_x_slide_array)!=0:
                    train_x_slide_array=np.vstack((train_x_slide_array,train_x_slide))
                else:
                    train_x_slide_array=np.ravel(train_x_slide)
            if len(train_x_array)!=0:
                train_x_array=np.vstack((train_x_array,np.ravel(train_x_slide_array)))
            else:
                train_x_array=np.ravel(train_x_slide_array)
            train_x_slide_array=np.array([],dtype=np.float)
    return train_x_array
    
train_x_array=slide(130)

## 预测1

In [57]:
train_y_array=np.load("data/rice.npy")

In [89]:
res={'colsample_bytree': 0.785758534325641, 'gamma': 0.0990589630344405, 'learning_rate': 0.09514632471525121, 'max_depth': 5, 'min_child_weight': 5.587679916099425, 'n_estimators': 145, 'reg_alpha': 0.005964585842081341, 'subsample': 0.8972563441534034}

In [105]:
res=xgboost_bayesian(train_x_array,
                 train_y_array
                ,probe=res,test=True,
                      test_size=.4
                 ,cv=3
                )

测试集: 2.0560897265570275


## 预测

In [36]:
train_x_array.shape

(243, 3672)

In [35]:
train_x_array=np.load("data/modified_direction.npy")
train_y_array=np.load("data/rice.npy")

In [59]:
import bayesian
from bayes_opt import BayesianOptimization
import xgboost

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [85]:
def xgboost_bayesian(data,target,probe=None,test=None,cv=3,test_size=.2):   # cv越大损失越大
    def rf_cv(learning_rate,n_estimators,max_depth,min_child_weight,gamma,subsample,colsample_bytree,reg_alpha):
#         kf = KFold(cv, shuffle=True, random_state=42).get_n_splits(X_train)
        xgboost_pipe=make_pipeline(StandardScaler(),xgboost.XGBRegressor(learning_rate=learning_rate,n_estimators=int(n_estimators),
                                                   max_depth=int(max_depth),
                                                   min_child_weight=min_child_weight,
                                                   gamma=gamma,subsample=subsample,
                                                   colsample_bytree=colsample_bytree,reg_alpha=reg_alpha,nthread=-1,n_jobs=-1))
        val = cross_val_score(xgboost_pipe,
                              X_train, y_train, scoring="neg_mean_squared_error", cv=cv).mean()
        return val
    X_train, X_test, y_train, y_test  =  train_test_split(data, target, test_size = test_size, random_state=10) #split the data

    if test!=None:
        predict=make_pipeline(StandardScaler(),xgboost.XGBRegressor(n_jobs=-1,**probe))
        predict.fit(X_train,y_train)
        loss = mean_squared_error(predict.predict(X_test), y_test)
        print("测试集:",loss)
        return probe
    
    bes=BayesianOptimization(rf_cv,{   
        'learning_rate': (0.05, 0.1),
        'n_estimators': (50, 500),
        'max_depth':[3,11],
        'min_child_weight':[1,6],
        "gamma":[0,1],
        'subsample':[0.4,1],
        'colsample_bytree':[0.75,1],
        'reg_alpha':[10**-5,10**-2],
         })

# {'colsample_bytree': 0.75, 
#  'gamma': 0.0, 
#  'learning_rate': 0.1, 
#  'max_depth': 3, 
#  'min_child_weight': 1.0,
#  'n_estimators': 500,
#  'reg_alpha': 0.01, 
#  'subsample': 0.4} 平滑后走最值
#     bes=BayesianOptimization(rf_cv,{
#         'learning_rate': (0.05, 0.5),
#         'n_estimators': (50, 1000),
#         'max_depth':[3,11],
#         'min_child_weight':[0.1,6],
#         "gamma":[0,1],
#         'subsample':[0.1,1],
#         'colsample_bytree':[0.25,1],
#         'reg_alpha':[10**-5,0.1],
#          })
    if probe!=None:
        bes.probe(probe)
    try:
        bes.maximize()
    except KeyboardInterrupt:
        res=bayesian.result_report(bes.res)
        print(res)
    finally:
        res=bayesian.result_report(bes.res)
        print(res)
        predict=make_pipeline(StandardScaler(),xgboost.XGBRegressor(**res))
        predict.fit(X_train,y_train)
        loss = mean_squared_error(predict.predict(X_test), y_test)
        print("测试集:",loss)
        return res

In [17]:
{'colsample_bytree': 0.785758534325641, 'gamma': 0.0990589630344405, 'learning_rate': 0.09514632471525121, 'max_depth': 5, 'min_child_weight': 5.587679916099425, 'n_estimators': 145, 'reg_alpha': 0.005964585842081341, 'subsample': 0.8972563441534034}
测试集: 0.8661445878527554

|   iter    |  target   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | reg_alpha | subsample |
-------------------------------------------------------------------------------------------------------------------------
|  1        | -3.539    |  1.0      |  0.0      |  0.05     |  3.0      |  6.0      |  468.0    |  0.01     |  1.0      |
|  2        | -3.529    |  0.7858   |  0.09906  |  0.09515  |  5.172    |  5.588    |  145.7    |  0.005965 |  0.8973   |
|  3        | -3.883    |  0.8375   |  0.789    |  0.07495  |  7.14     |  3.657    |  366.6    |  0.000677 |  0.9523   |
|  4        | -3.903    |  0.8123   |  0.0947   |  0.06325  |  4.531    |  3.89     |  313.4    |  0.005919 |  0.7659   |
|  5        | -3.806    |  0.8617   |  0.2307   |  0.09587  |  9.608    |  5.733    |  302.2    |  0.008839 |  0.9688   |
|  6        | -3.619    |  0.8576   |  0.7224   |  0.062    |  5.669    |  4.15     |  142.9    |  0.009153 |  0.6787   |
|  7        | -3.73     

In [92]:
res=xgboost_bayesian(train_x_array,
                 train_y_array
                ,probe=res,test_size=.4
                 ,cv=3
                )

|   iter    |  target   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | reg_alpha | subsample |
-------------------------------------------------------------------------------------------------------------------------
|  1        | -3.772    |  0.7858   |  0.09906  |  0.09515  |  5.0      |  5.588    |  145.0    |  0.005965 |  0.8973   |
|  2        | -3.746    |  0.8654   |  0.1213   |  0.05054  |  7.92     |  2.373    |  103.5    |  0.007493 |  0.6831   |
|  3        | -3.927    |  0.9614   |  0.9924   |  0.09483  |  3.994    |  5.888    |  259.4    |  0.001363 |  0.6769   |
|  4        | -3.615    |  0.8492   |  0.1144   |  0.06984  |  10.25    |  3.263    |  102.5    |  0.006521 |  0.7234   |
|  5        | -4.073    |  0.938    |  0.1711   |  0.06061  |  7.617    |  1.038    |  354.2    |  0.00946  |  0.9512   |
|  6        | -3.72     |  0.913    |  0.8355   |  0.09361  |  6.133    |  4.777    |  342.6    |  0.008202 |  0.8861   |
|  7        | -3.56     

In [68]:
predict=make_pipeline(StandardScaler(),xgboost.XGBRegressor(**res))
predict.fit(train_x_array,train_y_array)
predict_y=predict.predict(predict_x_array)

In [69]:
county=[]
for i in id_:
    county.append('county'+str(i))
    
predict_y=pd.DataFrame(data=predict_y,index=county)

In [90]:
predict_y

,0
county1,11.838021
county2,12.512785
county3,13.034824
county4,12.008980
county5,12.294268
county6,12.655515
county7,12.830249
county8,12.650788
county9,12.290071
county10,12.329312
